# Amazon Nova Act SDK を使用した基本的な Browser ツールの使い方

## 概要

このチュートリアルでは、Nova Act SDK を使用して Amazon Bedrock AgentCore Browser ツールを操作する方法を学びます。ヘッドレスでの Browser ツールの使用例とブラウザのライブビュー表示の例を紹介します。


### チュートリアル詳細


| 項目         | 詳細                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ       | 会話型                                                                   |
| エージェントタイプ          | シングル                                                                           |
| エージェントフレームワーク   | Nova Act                                                                         |
| LLM モデル           | Amazon Nova Act モデル                                                            |
| チュートリアルコンポーネント | Nova Act を使用したヘッドレスでの Browser ツール操作                    |
| チュートリアル分野   | 垂直統合                                                                         |
| 難易度  | 簡単                                                                             |
| 使用 SDK            | Amazon Bedrock AgentCore Python SDK、Nova Act                                     |

### チュートリアルアーキテクチャ

このチュートリアルでは、Nova Act と Browser ツールの使用方法を説明します。  

この例では、Nova Act エージェントに自然言語の指示を送り、Bedrock AgentCore Browser 上でヘッドレスでタスクを実行します。

### チュートリアルの主な機能

* ヘッドレスでの Browser ツールの使用
* Nova Act と Browser ツールの連携

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報。IAM ロール/ユーザーには以下の権限が必要です：https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html#browser-credentials-config
* Amazon Bedrock AgentCore SDK
* Nova Act SDK および API キー 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Bedrock AgentCore Browser ツールでの Nova Act の使用
ローカルで実行できる Python スクリプトを作成します。このスクリプトでは、Nova Act がブラウザセッションの CDP エンドポイントを使用して接続し、Playwright の操作を実行します。 

In [ ]:
%%writefile basic_browser_with_nova_act.py
"""Amazon Bedrock AgentCore と Nova Act を使用したブラウザ自動化スクリプト。

このスクリプトは、AI を活用した Web 自動化を実演します：
- Amazon Bedrock AgentCore を通じてブラウザセッションを初期化
- 自然言語による Web 操作のために Nova Act に接続
- ブラウザを使用した自動検索とデータ抽出を実行
"""

from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
from rich.console import Console
import argparse
import json

console = Console()

from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("使用リージョン:", region)

def browser_with_nova_act(prompt, starting_page, nova_act_key, region="us-west-2"):
    result = None
    with browser_session(region) as client:
        ws_url, headers = client.generate_ws_headers()
        try:
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                preview={"playwright_actuation": True},
                nova_act_api_key=nova_act_key,
                starting_page=starting_page,
            ) as nova_act:
                result = nova_act.act(prompt)
        except Exception as e:
            console.print(f"NovaAct エラー: {e}")
        finally:
            return result


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="ブラウザ検索の指示")
    parser.add_argument("--starting-page", required=True, help="開始 URL")
    parser.add_argument("--nova-act-key", required=True, help="Nova Act API キー")
    parser.add_argument("--region", default="us-west-2", help="AWS リージョン")
    args = parser.parse_args()

    result = browser_with_nova_act(
        args.prompt, args.starting_page, args.nova_act_key, args.region
    )
    console.print(f"\n[cyan] レスポンス[/cyan] {result.response}")
    console.print(f"\n[bold green]Nova Act 結果:[/bold green] {result}")

#### スクリプトの実行
スクリプトを実行する前に、以下に Nova Act API キーを貼り付けてください。 

In [ ]:
NOVA_ACT_KEY= '' ### Paste your Nova Act Key here

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Search for macbooks and extract the details of the first one" --starting-page "https://www.amazon.com/" --nova-act-key {NOVA_ACT_KEY}

In [ ]:
!python basic_browser_with_nova_act.py --prompt "Extract and return Amazon revenue for the last 4 years" --starting-page "https://stockanalysis.com/stocks/amzn/financials/" --nova-act-key {NOVA_ACT_KEY}

## 裏側で何が起きていたか？

* `browser_session` を使用してブラウザクライアントをインスタンス化すると、Browser クライアントが作成されセッションが開始されました
* その後、`cdp_endpoint_url` と `cdp_headers` を使用して、Nova Act をそのブラウザセッションに接続するよう設定しました
* Nova Act SDK は、あなたの自然言語の指示を受け取り、ブラウザ上で Playwright の操作を実行しました 

# おめでとうございます！